# Libraries

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

2024-03-24 22:50:31.305503: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 22:50:31.684732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 22:50:31.684787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 22:50:31.749281: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 22:50:31.883304: I tensorflow/core/platform/cpu_feature_guar

# Preprocessing

In [3]:
# base path
directory_path = './labels_final'
# current directory
c_dir = os.getcwd()

# all actions
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

# specific actions
# actions = np.array(['alligator', 'flower', 'kiss', 'listen', 'orange'])
# actions = np.array(['afternoon', 'house', 'again', 'open', 'kiss', 'sorry'])
# actions = np.array(sorted([folder for folder in os.listdir('./labels_new') if os.path.isdir(os.path.join(directory_path, folder))]))

In [4]:
# create a dictionary for int representation of actions
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'all': 0,
 'bye': 1,
 'close': 2,
 'down': 3,
 'for': 4,
 'have': 5,
 'hello': 6,
 'if': 7,
 'listen': 8,
 'mad': 9,
 'nap': 10,
 'no': 11,
 'noisy': 12,
 'now': 13,
 'please': 14,
 'quiet': 15,
 'sad': 16,
 'thankyou': 17,
 'time': 18,
 'will': 19}

Note that at this point, we will not access the video folder, only the numpy folder.

In [8]:
sequences, labels = [], []  # sequence -> video, labels -> action
for action in actions:
    no_actions = len(os.listdir(os.path.join(directory_path, action)))
    print('Opening path:', os.path.join(directory_path, action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []         # window -> single frame
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(directory_path, action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        if no_frames_per_action >= 30:
            for frame_num in range(1, no_frames_per_action + 1):
                res = np.load(os.path.join(directory_path, action, file,  "{}.npy".format(frame_num)))     # res -> coordinate key points
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])
    print('-'*75)

Opening path: ./labels_final/all
Number of instances: 386
Number of frames in all_1: 39
Number of frames in all_2: 33
Number of frames in all_3: 25
Number of frames in all_4: 11
Number of frames in all_5: 4
Number of frames in all_6: 7
Number of frames in all_7: 33
Number of frames in all_8: 87
Number of frames in all_9: 6
Number of frames in all_10: 29
Number of frames in all_11: 9
Number of frames in all_12: 133
Number of frames in all_13: 68
Number of frames in all_14: 11
Number of frames in all_15: 6
Number of frames in all_16: 46
Number of frames in all_17: 16
Number of frames in all_18: 18
Number of frames in all_19: 10
Number of frames in all_20: 56
Number of frames in all_21: 54
Number of frames in all_22: 9
Number of frames in all_23: 192
Number of frames in all_24: 31
Number of frames in all_25: 26
Number of frames in all_26: 30
Number of frames in all_27: 6
Number of frames in all_28: 33
Number of frames in all_29: 6
Number of frames in all_30: 55
Number of frames in all_31:

In [9]:
# due to difference in number of frames, pad x and y
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = to_categorical(labels).astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [10]:
input_shape = (x_train.shape[1], x_train.shape[2])
print(x_train.shape)
print(x_test.shape)

(2143, 341, 225)
(536, 341, 225)


In [12]:
np.save('x_train.npy', x_train)
np.save('x_test.npy', x_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

# base path
directory_path = './labels_final'
# current directory
c_dir = os.getcwd()

# all actions
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

input_shape = (x_train.shape[1], x_train.shape[2])

2024-03-25 23:01:27.609787: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 23:01:28.047193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 23:01:28.047240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 23:01:28.110238: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 23:01:28.245763: I tensorflow/core/platform/cpu_feature_guar

In [2]:
np.sum(y_train, axis = 0)

array([106, 129, 104,  75, 110,  83,  98, 107, 117, 102, 164, 100, 109,
        88, 158,  86, 124,  86, 111,  86])

# Models

## LSTM

In [21]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN, EarlyStopping

from sklearn.metrics import accuracy_score

In [22]:
# for logging of data with TensorBoard
log_dir = os.path.join(c_dir, 'Logs')
tb_callback = TensorBoard(log_dir = log_dir)

# to end training when failure happens ie. loss == nan
term = TerminateOnNaN()

# to stop training early if there is no change in loss
early = EarlyStopping(monitor = 'val_loss', patience = 200)

# regulariser 
l2_strength = 0.01

In [23]:
def choose_lstm(n):
    if n == 1:
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.1))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dropout(0.1))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 2:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = input_shape))
        model.add(LSTM(128, return_sequences = True))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dense(64))
        model.add(Dense(32))
        model.add(Dense(8))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 3:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.5))
        # model.add(Bidirectional(LSTM(64, return_sequences = True)))
        # model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(32)))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 4:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(32, return_sequences = True)))
        model.add(Dropout(0.3))
        model.add(Bidirectional(LSTM(16)))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 5:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(32, return_sequences = True)))
        model.add(Dropout(0.3))
        model.add(Bidirectional(LSTM(16, kernel_regularizer = keras.regularizers.L2(l2_strength))))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model
    
    elif n == 6:
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.3))
        model.add(Bidirectional(LSTM(32, kernel_regularizer = keras.regularizers.L2(l2_strength))))
        model.add(Dropout(0.3))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

In [24]:
model = choose_lstm(6)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 341, 128)          181248    
                                                                 
 dropout_4 (Dropout)         (None, 341, 128)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 20)                1300      
                                                                 
Total params: 223764 (874.08 KB)
Trainable params: 223764 (874.08 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [25]:
model.fit(x_train, y_train, epochs = 800, batch_size = 32, validation_split = 0.2, callbacks = [term, tb_callback, early])

Epoch 1/800
54/54 [==============================] - 8s 90ms/step - loss: 4.6605 - categorical_accuracy: 0.0636 - val_loss: 3.9190 - val_categorical_accuracy: 0.0559
Epoch 2/800
54/54 [==============================] - 3s 56ms/step - loss: 3.5593 - categorical_accuracy: 0.0747 - val_loss: 3.3029 - val_categorical_accuracy: 0.0862
Epoch 3/800
54/54 [==============================] - 3s 55ms/step - loss: 3.1687 - categorical_accuracy: 0.1009 - val_loss: 3.0824 - val_categorical_accuracy: 0.1096
Epoch 4/800
54/54 [==============================] - 3s 56ms/step - loss: 3.0266 - categorical_accuracy: 0.1126 - val_loss: 3.0062 - val_categorical_accuracy: 0.0816
Epoch 5/800
54/54 [==============================] - 3s 56ms/step - loss: 2.9816 - categorical_accuracy: 0.0974 - val_loss: 2.9575 - val_categorical_accuracy: 0.1166
Epoch 6/800
54/54 [==============================] - 3s 56ms/step - loss: 2.9344 - categorical_accuracy: 0.1155 - val_loss: 2.9796 - val_categorical_accuracy: 0.0956
Epoc

In [26]:
# take model predictions
res = model.predict(x_test)

17/17 [==============================] - 2s 28ms/step


In [27]:
res

array([[4.1061123e-09, 7.5535063e-09, 2.0438514e-04, ..., 5.4142212e-05,
        4.2355073e-06, 3.2292348e-07],
       [4.4064935e-05, 3.9149035e-04, 6.3190993e-05, ..., 5.9357874e-10,
        4.9074675e-08, 7.1135275e-02],
       [2.0059696e-04, 6.1231901e-07, 1.1068702e-04, ..., 7.0037320e-04,
        1.3316007e-05, 3.6237521e-05],
       ...,
       [3.7374977e-02, 1.0055536e-03, 2.5104406e-01, ..., 2.5454061e-02,
        1.2132803e-01, 1.0973195e-02],
       [1.3520292e-02, 1.1444185e-03, 9.2925859e-04, ..., 1.6117521e-07,
        9.3189474e-06, 9.5859289e-01],
       [2.1961822e-04, 4.5971880e-03, 7.4904063e-05, ..., 9.6191499e-10,
        7.1945954e-07, 2.1956411e-01]], dtype=float32)

In [28]:
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred = np.argmax(res, axis = 1).tolist()

In [29]:
print(y_true)
print(y_pred)

[9, 6, 14, 18, 1, 19, 3, 4, 0, 12, 10, 4, 16, 7, 17, 8, 19, 10, 12, 5, 1, 0, 11, 1, 8, 15, 2, 14, 0, 4, 10, 7, 1, 1, 10, 4, 1, 14, 10, 11, 12, 11, 10, 7, 11, 1, 18, 0, 15, 7, 13, 14, 6, 13, 10, 7, 15, 10, 12, 10, 6, 11, 4, 0, 12, 10, 3, 16, 13, 12, 9, 6, 9, 6, 8, 16, 4, 4, 12, 1, 9, 2, 14, 15, 19, 11, 4, 10, 7, 6, 17, 7, 14, 12, 18, 15, 16, 3, 18, 6, 9, 19, 14, 10, 17, 6, 7, 10, 8, 16, 8, 14, 4, 14, 13, 4, 3, 17, 2, 10, 10, 12, 9, 2, 13, 11, 11, 16, 9, 15, 6, 8, 13, 4, 1, 1, 14, 2, 2, 3, 16, 6, 10, 4, 9, 10, 12, 14, 0, 11, 1, 4, 5, 14, 19, 9, 16, 9, 0, 12, 7, 19, 2, 11, 4, 14, 1, 13, 18, 13, 4, 2, 15, 4, 10, 10, 11, 11, 10, 13, 16, 2, 8, 10, 0, 10, 1, 15, 11, 14, 0, 14, 2, 16, 8, 0, 6, 8, 3, 14, 7, 8, 18, 9, 4, 10, 4, 2, 18, 8, 8, 12, 3, 17, 9, 2, 10, 1, 13, 9, 16, 13, 9, 10, 19, 19, 11, 12, 14, 7, 16, 19, 17, 4, 1, 8, 1, 17, 0, 2, 14, 10, 14, 0, 16, 5, 10, 0, 11, 13, 0, 18, 4, 16, 16, 8, 0, 16, 5, 13, 6, 0, 3, 2, 8, 13, 15, 18, 10, 0, 2, 6, 0, 3, 17, 19, 16, 3, 12, 12, 1, 5, 14, 8, 14

In [30]:
print(f'Accuracy: {round(accuracy_score(y_true, y_pred)*100, 1)}%')

Accuracy: 54.7%


In [31]:
if accuracy_score(y_true, y_pred) >= 0.70:
    model.save('lstm_model_final.h5', save_format = 'h5')
else:
    print('Model accuracy insufficient')

# keras.models.load_model("test_model.keras")

Model accuracy insufficient


# Hyperparameter Tuning (Cross-validation)

In [32]:
# from scikeras.wrappers import KerasClassifier
# from sklearn.model_selection import GridSearchCV

In [33]:
# def create_lstm_model(units = 64, dropout_rate = 0.5, l2_strength = 0.01, learning_rate=0.001):
#     model = Sequential()
#     model.add(LSTM(units, return_sequences = True, input_shape = input_shape))
#     model.add(Dropout(dropout_rate))
#     model.add(Bidirectional(LSTM(units, return_sequences = True)))
#     model.add(Dropout(dropout_rate))
#     model.add(Bidirectional(LSTM(units)))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(actions.shape[0], activation = "softmax"))
    
#     opt = keras.optimizers.Adam(learning_rate = learning_rate)
#     model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
    
#     return model

# lstm_model = KerasClassifier(build_fn = create_lstm_model, epochs = 800, batch_size = 32, validation_split = 0.2, verbose = 1)

# # hyperparameters
# param_grid = {
#     'units': [64, 128],  
#     'dropout_rate': [0.2, 0.5],  
#     'l2_strength': [0.001, 0.01], 
#     'learning_rate': [0.001, 0.01]   
# }

# # grid search
# grid = GridSearchCV(estimator=lstm_model, param_grid=param_grid, cv=3, verbose=2)
# grid_result = grid.fit(x_train, y_train, callbacks = [term, tb_callback, early]) 

# # best parameters and best accuracy
# print("Best parameters found: ", grid_result.best_params_)
# print("Best accuracy found: ", grid_result.best_score_)